# Add document-level metadata

This temporary notebook combines additional document level metadata from CCLW with the dataset in ```processed_policies``` that has already been extracted from CCLW.

In [253]:
from pathlib import Path

import pandas as pd
from bs4 import BeautifulSoup

## Load datasets

In [254]:
data_path = Path("../../data")
cpr_policies_df = pd.read_csv(data_path / "processed_policies.csv")
cclw_df = pd.read_csv(
    data_path / "cclw" / "cclw_policies_fe_26102021.csv",
    usecols=["Title", "Geography ISO", "Instruments", "Natural Hazards", "Responses", "Events", "Description"]
)

# Rename cclw columns
cclw_df.rename(columns={
    "Title": "policy_name",
    "Geography ISO": "country_code", 
    "Instruments": "instrument_list", 
    "Natural Hazards": "hazard_list", 
    "Responses": "responses_list", 
    "Events": "events_list", 
    "Description": "policy_description"
}, inplace=True)

Check counts to make sure we don't lose any rows from merging

In [255]:
len(cpr_policies_df), len(cclw_df)

(7121, 2382)

Verify how many cclw policies are in the cpr dataset

In [256]:
len(cpr_policies_df.loc[cpr_policies_df.source_name == "cclw"])

2418

## Merge datasets

Get cpr policies that were originally loaded from CCLW

In [257]:
cpr_policies_source_cclw_df = cpr_policies_df.loc[cpr_policies_df.source_name == "cclw"]

Remove any duplicates in the cclw dataset (this may result in loss of some data, but prevents duplicates being created in the merge)

In [258]:
cclw_df.drop_duplicates(subset=["policy_name", "country_code"], inplace=True)

Merge the cclw dataset with cpr. This is a left join since there are cpd policies in the cpr dataset too and we don't want to lose them.

In [259]:
cpr_cclw_policies_df = pd.merge(
    cpr_policies_source_cclw_df,
    cclw_df,
    on=["policy_name", "country_code"],
    how="left"
)

In [260]:
len(cpr_cclw_policies_df)

2418

## Prepare metadata

Process metadata fields from cclw:

- Create semi-colon delimited lists for columns containing multiple values: responses_list, instruments_list, hazards_list
- Create a policy_date column from the first date in the events_list column

### Instruments

Each entry for instruments is a semi-colon separated list with sub-instrument and top level instrument for each entry:

```
instrument_name|instrument_category; instrument_name|instrument_category; ...
```

We will extract the ```instrument_name``` for each entry and keep as a semi-colon separated list.

In [261]:
def extract_instr_list(instruments: str):
    if not pd.isnull(instruments):
        processed_instr_list = []
        instr_list = instruments.split(";")
        for instr in instr_list:
            processed_instr_list.append(
                instr.split("|")[0].strip()
            )
        
        return ";".join(processed_instr_list)
    else:
        return instruments

In [262]:
extract_instr_list("Monitoring and evaluation|Governance and planning; Developing plans and strategies|Governance and planning;  Capacity-building - general|Capacity-building")

'Monitoring and evaluation;Developing plans and strategies;Capacity-building - general'

In [263]:
cpr_cclw_policies_df.loc[:, "instrument_list"] = cpr_cclw_policies_df.loc[:, "instrument_list"].apply(lambda i: extract_instr_list(i))

### Responses

In [264]:
def remove_list_spaces(cclw_list, sep=","):
    if not pd.isnull(cclw_list):
        cclw_list = cclw_list.split(sep)
    
        return ";".join([cclw_item.strip() for cclw_item in cclw_list])
    else:
        return cclw_list

In [265]:
cpr_cclw_policies_df.loc[:, "responses_list"] = cpr_cclw_policies_df.loc[:, "responses_list"].apply(lambda i: remove_list_spaces(i))

### Hazards

In [266]:
cpr_cclw_policies_df.loc[:, "hazard_list"] = cpr_cclw_policies_df.loc[:, "hazard_list"].apply(lambda i: remove_list_spaces(i))

### Events

Events in cclw are a semicolon separated list of events, in the following format:

```
event_date|event_name; event_date|event_name; ...
```

We will extract the first event date in the list.

In [267]:
def extract_create_event(events: str):
    if not pd.isnull(events):
        processed_event_list = []
        instr_list = events.split(";")
        for instr in instr_list:
            processed_event_list.append(
                instr.split("|")[0].strip()
            )
        
        return processed_event_list[0]
    else:
        return events

In [268]:
cpr_cclw_policies_df["policy_date"] = None
cpr_cclw_policies_df.loc[:, "policy_date"] = cpr_cclw_policies_df.loc[:, "events_list"].apply(lambda e: extract_create_event(e))

Check policy_date is in the correct format

In [269]:
cpr_cclw_policies_df.policy_date.str.len().unique()

array([10., nan])

All policy_date values are either 10 digits (dd/mm/yyyy) or missing, so looks good.

In [292]:
del cpr_cclw_policies_df["events_list"]

## Map cclw sector and instruments to cpr definitions

Load instrument and sector mappings

In [270]:
schema_path = Path("../../schema")

In [271]:
instrument_mapping = pd.read_csv(schema_path / "cclw_instrument_mapping.csv", index_col=False)
sector_mapping = pd.read_csv(schema_path / "cclw_sector_mapping.csv", index_col=False)

In [272]:
instrument_mapping = instrument_mapping.dropna().set_index("instrument_cclw")["instrument_cpr"].to_dict()
sector_mapping = sector_mapping.dropna().set_index("sector_cclw")["sector_cpr"].to_dict()

In [273]:
NO_MAPPING = "[no_mapping]"

def map_cclw_values(cclw_value_list, mapping, sep=";"):
    if not pd.isnull(cclw_value_list):
        cclw_value_list = cclw_value_list.split(sep)
        cclw_value_list = [mapping.get(cclw_value, NO_MAPPING) for cclw_value in cclw_value_list]

        return sep.join(cclw_value_list)
    else:
        return cclw_value_list

In [274]:
cpr_cclw_policies_df.loc[:, "new_sector_list"] = cpr_cclw_policies_df.loc[:, "sector_list"].apply(lambda v: map_cclw_values(v, sector_mapping))
cpr_cclw_policies_df.loc[:, ["sector_list", "new_sector_list"]].head(5)

,sector_list,new_sector_list
0,Industry;Agriculture;Economy-wide,Industry;Agriculture (general);Economy-wide
1,Transport;Rural;LULUCF;Energy;Economy-wide;Agr...,"Transport (general);Rural;Land use, land use c..."
2,Transport;Rural;LULUCF;Energy;Economy-wide;Agr...,"Transport (general);Rural;Land use, land use c..."
3,LULUCF,"Land use, land use change and forestry (LULUCF)"
4,LULUCF,"Land use, land use change and forestry (LULUCF)"


In [275]:
mapping_mask = cpr_cclw_policies_df.new_sector_list.str.contains("no_mapping")
mapping_mask.fillna(False, inplace=True)
missing_sector_mappings = cpr_cclw_policies_df.loc[mapping_mask, ["sector_list", "new_sector_list"]].drop_duplicates(subset=["sector_list"])

In [276]:
def get_missing_mappings(missing_mappings, cclw_col, mapped_col, sep=";"):
    missing = []

    for row_ix, row in missing_mappings.iterrows():
        cclw = row[cclw_col].split(sep)
        mapped = row[mapped_col].split(sep)

        missing_cclw = []
        missing_mapped = []

        for mapped_item_ix, mapped in enumerate(mapped):
            if mapped == "[no_mapping]":
                missing_cclw.append(cclw[mapped_item_ix])
                missing_mapped.append(mapped)

        if len(missing_cclw) > 0:
            missing += list(zip(missing_cclw, missing_mapped))

    return set(missing)

In [277]:
get_missing_mappings(missing_sector_mappings, "sector_list", "new_sector_list")

set()

In [278]:
cpr_cclw_policies_df.loc[:, "new_instrument_list"] = cpr_cclw_policies_df.loc[:, "instrument_list"].apply(lambda v: map_cclw_values(v, instrument_mapping))
cpr_cclw_policies_df.loc[:, ["instrument_list", "new_instrument_list"]].sample(5)

,instrument_list,new_instrument_list
1355,Developing plans and strategies,Developing plans and strategies
784,Creating bodies and institutions;Designing pro...,Creating bodies and institutions;Designing pro...
2333,Creating bodies and institutions,Creating bodies and institutions
635,Subsidies;Designing processes;Taxes,Fiscal or financial incentives;Designing proce...
1707,Developing plans and strategies;Multi-level go...,Developing plans and strategies;Multi-level go...


In [279]:
mapping_mask = cpr_cclw_policies_df.new_instrument_list.str.contains("no_mapping")
mapping_mask.fillna(False, inplace=True)
missing_instr_mappings = cpr_cclw_policies_df.loc[mapping_mask, ["instrument_list", "new_instrument_list"]].drop_duplicates(subset=["instrument_list"])

In [280]:
get_missing_mappings(missing_instr_mappings, "instrument_list", "new_instrument_list")

set()

In [281]:
del cpr_cclw_policies_df["sector_list"]
del cpr_cclw_policies_df["instrument_list"]
cpr_cclw_policies_df.rename(columns={
    "new_sector_list": "sector_list",
    "new_instrument_list": "instrument_list"
}, inplace=True)

### Policy description

Policy description from cclw contains html markup. We will use BeautifulSoup to extract only the text for this field.

In [284]:
def get_description_text(description):
    if not pd.isnull(description):
        desc_html_parser = BeautifulSoup(description, "html.parser")
        return desc_html_parser.text
    else:
        return description

In [287]:
cpr_cclw_policies_df.loc[:, "policy_description"] = cpr_cclw_policies_df.loc[:, "policy_description"].apply(lambda d: get_description_text(d))

## Prepare final dataset

Merge cclw and cpd policies to create final dataset

In [293]:
merged = pd.concat([
    cpr_cclw_policies_df,
    cpr_policies_df.loc[cpr_policies_df.source_name == "cpd", :]
], axis=0)

In [294]:
merged.to_csv(data_path / "processed_policies_merged.csv")